In [69]:
import pandas as pd
import numpy as np

In [3]:
tracking_test = pd.read_json("Player_Tracking/CLEBOS/2017051902_nba-bos_TRACKING.jsonl.gz", lines=True,
                             compression='gzip', encoding='latin1')

In [12]:
tracking_test.head()

,awayPlayers,ball,frameIdx,gameClock,gameClockStopped,homePlayers,period,shotClock,wallClock
0,[{'playerId': 'ff41e9f0-89ef-11e6-87b7-a45e60e...,"{'xyz': [1.56, 8.88, 3.52]}",0,720.0,True,[{'playerId': 'ff427c2e-89ef-11e6-8d2a-a45e60e...,1,24.0,1495240341536
1,[{'playerId': 'ff41e9f0-89ef-11e6-87b7-a45e60e...,"{'xyz': [0.64, 8.06, 4.49]}",1,720.0,True,[{'playerId': 'ff427c2e-89ef-11e6-8d2a-a45e60e...,1,24.0,1495240341576
2,[{'playerId': 'ff41e9f0-89ef-11e6-87b7-a45e60e...,"{'xyz': [0.59, 8.22, 4.28]}",2,720.0,True,[{'playerId': 'ff427c2e-89ef-11e6-8d2a-a45e60e...,1,24.0,1495240341616
3,[{'playerId': 'ff41e9f0-89ef-11e6-87b7-a45e60e...,"{'xyz': [1.25, 8.72, 3.77]}",3,720.0,True,[{'playerId': 'ff427c2e-89ef-11e6-8d2a-a45e60e...,1,24.0,1495240341656
4,[{'playerId': 'ff41e9f0-89ef-11e6-87b7-a45e60e...,"{'xyz': [1.4, 8.69, 3.79]}",4,720.0,True,[{'playerId': 'ff427c2e-89ef-11e6-8d2a-a45e60e...,1,24.0,1495240341696


In [58]:
events_test = pd.read_json("Player_Tracking/CLEBOS/2017051902_nba-bos_EVENTS.jsonl", lines=True)

In [35]:
events_test.eventType.unique()

array(['TOUCH', 'DRIBBLE', 'PASS', 'SHOT', 'REB', 'FOUL', 'TO', 'FT',
       'VIO', 'TMO'], dtype=object)

In [59]:
temp = tracking_test.merge(events_test[['eventType','playerId','wallClock','gameClock']], how='left', on=['wallClock','gameClock'])

In [61]:
temp.dropna(subset=['eventType','playerId'], how='all', inplace=True)

In [62]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3603 entries, 147 to 84892
Data columns (total 11 columns):
awayPlayers         3603 non-null object
ball                3603 non-null object
frameIdx            3603 non-null int64
gameClock           3603 non-null float64
gameClockStopped    3603 non-null bool
homePlayers         3603 non-null object
period              3603 non-null int64
shotClock           3549 non-null float64
wallClock           3603 non-null int64
eventType           3603 non-null object
playerId            3574 non-null object
dtypes: bool(1), float64(2), int64(3), object(5)
memory usage: 313.2+ KB


In [24]:
temp.loc[0,'awayPlayers']

[{'playerId': 'ff41e9f0-89ef-11e6-87b7-a45e60e298d3',
  'xyz': [-24.66, -5.57, 0.0]},
 {'playerId': 'ff403f7d-89ef-11e6-b44d-a45e60e298d3',
  'xyz': [1.47, 11.13, 0.0]},
 {'playerId': 'ff42aecf-89ef-11e6-8cb8-a45e60e298d3',
  'xyz': [-10.35, 1.3599999999999999, 0.0]},
 {'playerId': 'ff4221b5-89ef-11e6-b799-a45e60e298d3',
  'xyz': [-25.3, 1.9500000000000002, 0.0]},
 {'playerId': 'ff41f1fa-89ef-11e6-a048-a45e60e298d3',
  'xyz': [-1.15, -0.37, 0.0]}]

In [103]:
def label_half_court(ss_df):
    """
    Labels which half-court the possession happens in.
    0 for left half-court. 1 for right half-court.
    Also trims the possession time-series data to only
    include data of front-court. Back-court data points
    are omitted. Our visualization only cares about 
    front-court possessions.
    
    Args:
        ss_df: secondspectrum tracking data dataframe,
               is modified inplace to have column half_court_idx
               
    """
    ss_df['half_court_idx'] = -1
    ss_df['half_court_idx'] = ss_df['half_court_idx'].astype(int)
    
    
    teamA, teamB = -1, -1
    halfcourtA, halfcourtB = -1, -1
    
    # find the first "SHOT" event, corresponding team & front-court
    for index, row in ss_df.iterrows():
        if row['eventType'] == 'SHOT':
            break
    teamA = row['poss_team']
    halfcourtA = 0 if row['ball']['xyz'][0] < 0 else 1
    halfcourtB = int(not halfcourtA)
    
    
    halfcourt = halfcourtA
    poss_team = teamA
    for index, row in ss_df.iterrows():
        if row['period'] > 2:
            halfcourtA = int(not halfcourtA)
            halfcourtB = int(not halfcourtB)
        if row['poss_team'] != poss_team:
            halfcourt = halfcourtB if halfcourt == halfcourtA else halfcourtA
            poss_team = row['poss_team']
        
        ss_df.at[index, 'half_court_idx'] = halfcourt

In [104]:
label_half_court(temp)

In [106]:
temp.head(200)

,awayPlayers,ball,frameIdx,gameClock,gameClockStopped,homePlayers,period,shotClock,wallClock,eventType,playerId,poss_player,poss_team,poss_idx,half_court_idx
147,{'ff41e9f0-89ef-11e6-87b7-a45e60e298d3': [-8.5...,"{'xyz': [20.29, -1.72, 6.52]}",147,719.11,False,{'ff427c2e-89ef-11e6-8d2a-a45e60e298d3': [-0.5...,1,24.00,1495240347417,TOUCH,ff413f45-89ef-11e6-94c9-a45e60e298d3,ff413f45-89ef-11e6-94c9-a45e60e298d3,0,1,0
175,{'ff41e9f0-89ef-11e6-87b7-a45e60e298d3': [-15....,"{'xyz': [18.04, 0.75, 0.21]}",175,717.99,False,{'ff427c2e-89ef-11e6-8d2a-a45e60e298d3': [-11....,1,23.15,1495240348536,DRIBBLE,ff413f45-89ef-11e6-94c9-a45e60e298d3,ff413f45-89ef-11e6-94c9-a45e60e298d3,0,1,0
192,{'ff41e9f0-89ef-11e6-87b7-a45e60e298d3': [-18....,"{'xyz': [14.32, 0.18, 0.36]}",192,717.31,False,{'ff427c2e-89ef-11e6-8d2a-a45e60e298d3': [-20....,1,22.48,1495240349216,DRIBBLE,ff413f45-89ef-11e6-94c9-a45e60e298d3,ff413f45-89ef-11e6-94c9-a45e60e298d3,0,1,0
212,{'ff41e9f0-89ef-11e6-87b7-a45e60e298d3': [-21....,"{'xyz': [11.46, 0.15, 0.13]}",212,716.50,False,{'ff427c2e-89ef-11e6-8d2a-a45e60e298d3': [-27....,1,21.69,1495240350016,DRIBBLE,ff413f45-89ef-11e6-94c9-a45e60e298d3,ff413f45-89ef-11e6-94c9-a45e60e298d3,0,1,0
289,{'ff41e9f0-89ef-11e6-87b7-a45e60e298d3': [-44....,"{'xyz': [-4.55, -0.09, 0.28]}",289,713.42,False,{'ff427c2e-89ef-11e6-8d2a-a45e60e298d3': [-28....,1,18.65,1495240353096,DRIBBLE,ff413f45-89ef-11e6-94c9-a45e60e298d3,ff413f45-89ef-11e6-94c9-a45e60e298d3,0,1,0
315,{'ff41e9f0-89ef-11e6-87b7-a45e60e298d3': [-42....,"{'xyz': [-9.18, -7.46, 0.29]}",315,712.38,False,{'ff427c2e-89ef-11e6-8d2a-a45e60e298d3': [-23....,1,17.62,1495240354136,DRIBBLE,ff413f45-89ef-11e6-94c9-a45e60e298d3,ff413f45-89ef-11e6-94c9-a45e60e298d3,0,1,0
330,{'ff41e9f0-89ef-11e6-87b7-a45e60e298d3': [-40....,"{'xyz': [-12.86, -15.98, 0.49]}",330,711.77,False,{'ff427c2e-89ef-11e6-8d2a-a45e60e298d3': [-22....,1,17.03,1495240354736,DRIBBLE,ff413f45-89ef-11e6-94c9-a45e60e298d3,ff413f45-89ef-11e6-94c9-a45e60e298d3,0,1,0
344,{'ff41e9f0-89ef-11e6-87b7-a45e60e298d3': [-39....,"{'xyz': [-19.48, -19.28, 0.8]}",344,711.21,False,{'ff427c2e-89ef-11e6-8d2a-a45e60e298d3': [-27....,1,16.47,1495240355296,DRIBBLE,ff413f45-89ef-11e6-94c9-a45e60e298d3,ff413f45-89ef-11e6-94c9-a45e60e298d3,0,1,0
371,{'ff41e9f0-89ef-11e6-87b7-a45e60e298d3': [-38....,"{'xyz': [-26.53, -15.47, 0.67]}",371,710.13,False,{'ff427c2e-89ef-11e6-8d2a-a45e60e298d3': [-38....,1,15.41,1495240356376,DRIBBLE,ff413f45-89ef-11e6-94c9-a45e60e298d3,ff413f45-89ef-11e6-94c9-a45e60e298d3,0,1,0
383,{'ff41e9f0-89ef-11e6-87b7-a45e60e298d3': [-38....,"{'xyz': [-22.48, -14.94, 4.06]}",383,709.65,False,{'ff427c2e-89ef-11e6-8d2a-a45e60e298d3': [-40....,1,14.93,1495240356856,PASS,ff413f45-89ef-11e6-94c9-a45e60e298d3,ff413f45-89ef-11e6-94c9-a45e60e298d3,0,1,0
